In [5]:
import numpy as np
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight

In [2]:
df = pd.read_csv('mlb_dataset.csv')

df.shape

(48930, 15)

In [7]:
df['hate'].value_counts()

hate
1    29046
0    19884
Name: count, dtype: int64